In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import hist
from collections import defaultdict
import mplhep as hep
hep.style.use('CMS')

### Run info

In [ ]:
run_info = {
    ### Offset 20
    'Run 0':{
        'start': datetime.datetime(2024, 6, 5, 0, 19),
        'duration': 1,
    },
    'Run 1':{
        'start': datetime.datetime(2024, 6, 5, 2, 20),
        'duration': 2,
    },
    'Run 2':{
        'start': datetime.datetime(2024, 6, 5, 10, 18),
        'duration': 2,
    },
    'Run 3':{
        'start': datetime.datetime(2024, 6, 5, 12, 35),
        'duration': 2,
    },
    'Run 4':{
        'start': datetime.datetime(2024, 6, 5, 17, 3),
        'duration': 2,
    },
    'Run 5':{
        'start': datetime.datetime(2024, 6, 5, 19, 38),
        'duration': 1,
    },
    'Run 6':{
        'start': datetime.datetime(2024, 6, 5, 21, 8),
        'duration': 2,
    },
    'Run 7':{
        'start': datetime.datetime(2024, 6, 5, 23, 25),
        'duration': 1,
    },
    'Run 8':{
        'start': datetime.datetime(2024, 6, 6, 1, 2),
        'duration': 3,
    },
    'Run 9':{
        'start': datetime.datetime(2024, 6, 6, 10, 47),
        'duration': 2,
    },
    'Run 10':{
        'start': datetime.datetime(2024, 6, 6, 13, 25),
        'duration': 2,
    },
    'Run 11':{
        'start': datetime.datetime(2024, 6, 6, 15, 49),
        'duration': 2,
    },
    'Run 12':{
        'start': datetime.datetime(2024, 6, 6, 18, 20),
        'duration': 2,
    },
    'Run 13':{
        'start': datetime.datetime(2024, 6, 6, 20, 39),
        'duration': 2,
    },
    'Run 14':{
        'start': datetime.datetime(2024, 6, 6, 22, 58),
        'duration': 2,
    },
    'Run 15':{
        'start': datetime.datetime(2024, 6, 7, 1, 28),
        'duration': 3,
    },
}

In [ ]:
con = sqlite3.connect('/media/daq/X9/DESYJune2024/ETROC-History/BaselineHistory.sqlite')
df = pd.read_sql_query("SELECT * from baselines", con)

In [ ]:
interest_df = df.loc[(df['col']==8) & (df['row']==11)].reset_index(drop=True)
# interest_df

In [ ]:
chip_names_list = interest_df['chip_name'].unique()
print(chip_names_list)
boards_of_interest = ['ET2p01_Bar4', 'ET2p01_Bar12', 'ET2p01_Bar13', 'ET2p01_CNM13HPK']

### Select data

In [ ]:
interest_df['timestamp'] = pd.to_datetime(interest_df['timestamp'])

selected_baselines = {}

for key, val in run_info.items():
    cut_start_time = val['start'] - datetime.timedelta(hours=0, minutes=30)
    cut_end_time = cut_start_time + datetime.timedelta(hours=val['duration'])
    filtered_df = interest_df.loc[(interest_df['timestamp'] > cut_start_time) & (interest_df['timestamp'] < cut_end_time)]
    selected_baselines[key] = {}
    for iboard in boards_of_interest:
        b_filtered_df = filtered_df.loc[(filtered_df['chip_name'] == iboard)]
        selected_baselines[key][iboard] = b_filtered_df['baseline'].astype('int32').values

# interest_df['timestamp'] = pd.to_datetime(interest_df['timestamp'])
# interest_df = interest_df.loc[(interest_df['timestamp'] > run_info['Run32']['start'] -  datetime.timedelta(hours=1))]

In [ ]:
selected_baselines

In [ ]:
# for k,iboard in enumerate(boards_of_interest):
#     x_labels = list(selected_baselines.keys())
#     x_range = np.arange(0, len(x_labels))
#     y_vals = []
#     for key, val in selected_baselines.items():
#         try:
#             y_vals.append(val[iboard][0])
#         except:
#             y_vals.append(np.nan)

#     fig, ax = plt.subplots(figsize=(11, 11))
#     ax.plot(x_range, y_vals, marker='o')
#     ax.set_title(iboard, loc='right', fontsize=18)
#     ax.set_xticks(x_range)
#     ax.set_xticklabels(x_labels, rotation=90)
#     ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True)) # Setting y-ticks to display only integers
#     plt.minorticks_off()
#     ax.grid()
#     plt.tight_layout()

### Baseline over time

In [ ]:
chip_names_list = interest_df['chip_name'].unique()
print(chip_names_list)

In [ ]:
draw_dates = True

fig, ax = plt.subplots(figsize=(20, 10))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
ax.set_title('Baseline of pixel (8, 11)', size=17, loc="right")

chip_names_list = interest_df['chip_name'].unique()

for idx, iboard in enumerate(['ET2p01_Bar4','ET2p01_Bar12','ET2p01_Bar13','ET2p01_CNM13HPK']):
    # if idx == 0: continue
    tmp_df = interest_df.loc[interest_df['chip_name']==iboard]
    tmp_df['timestamp'] = pd.to_datetime(tmp_df['timestamp'])
    tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-05-01 22:00:00')]
    ax.plot(tmp_df['timestamp'], tmp_df['baseline'], label=f'{iboard}', marker='.')

    if idx == 0:
        date_form = mdates.DateFormatter("%m-%d\n%H:%M")
        ax.xaxis.set_major_formatter(date_form)

if draw_dates:
    for irun, itime in run_info.items():
        start_time = itime['start']
        end_time = start_time + datetime.timedelta(hours=itime['duration'])
        text_start_time = start_time #- datetime.timedelta(hours=1)
        text_end_time = end_time #+ datetime.timedelta(hours=1)
        plt.axvline(x=pd.to_datetime(start_time), color='magenta', linestyle='dashed')
        ax.text(x=pd.to_datetime(text_start_time), y=400, s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
        plt.axvline(x=pd.to_datetime(end_time), color='black', linestyle='dashdot')
        ax.text(x=pd.to_datetime(text_end_time), y=400, s=f'{irun} End', rotation=90, va='center', fontsize=12)

ax.legend(loc='center left')

In [ ]:
pixel_plot_list_row,pixel_plot_list_col = np.meshgrid([2, 5, 7,8,9, 13], [2,5,7,8,9,13])
pixel_plot_list = list(zip(pixel_plot_list_row.flatten(),pixel_plot_list_col.flatten()))
for idx, iboard in enumerate(['ET2p01_Bar4','ET2p01_Bar12','ET2p01_Bar13','ET2p01_CNM13HPK']):
    fig, ax = plt.subplots(figsize=(20, 10))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
    ax.set_title(f'Baseline of board {iboard}', size=17, loc="right")
    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)
    for row,col in pixel_plot_list:
        interest_df = df.loc[(df['col']==col) & (df['row']==row)].reset_index(drop=True)
        tmp_df = interest_df.loc[interest_df['chip_name']==iboard]
        tmp_df['timestamp'] = pd.to_datetime(tmp_df['timestamp'])
        tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-05-01 22:00:00')]
        ax.plot(tmp_df['timestamp'], tmp_df['baseline'], label=f'Pixel R{row} C{col}', marker='.', c='r' if col<8 else 'b')
        # ax.legend()
    for irun, itime in run_info.items():
        start_time = itime['start']
        end_time = start_time + datetime.timedelta(hours=itime['duration'])
        text_start_time = start_time #- datetime.timedelta(hours=1)
        text_end_time = end_time #+ datetime.timedelta(hours=1)
        plt.axvline(x=pd.to_datetime(start_time), color='magenta', linestyle='dashed')
        # ax.text(x=pd.to_datetime(text_start_time), y=400, s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
        plt.axvline(x=pd.to_datetime(end_time), color='black', linestyle='dashdot')
        # ax.text(x=pd.to_datetime(text_end_time), y=400, s=f'{irun} End', rotation=90, va='center', fontsize=12)

In [ ]:
pixel_plot_list_row,pixel_plot_list_col = np.meshgrid([2, 5, 7,8,9, 13], [2,5,7,8,9,13])
pixel_plot_list = list(zip(pixel_plot_list_row.flatten(),pixel_plot_list_col.flatten()))
for idx, iboard in enumerate(['ET2p01_Bar4','ET2p01_Bar12','ET2p01_Bar13','ET2p01_CNM13HPK']):
    fig, ax = plt.subplots(figsize=(20, 10))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
    ax.set_title(f'Noise Width of board {iboard}', size=17, loc="right")
    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)
    for row,col in pixel_plot_list:
        interest_df = df.loc[(df['col']==col) & (df['row']==row)].reset_index(drop=True)
        tmp_df = interest_df.loc[interest_df['chip_name']==iboard]
        tmp_df['timestamp'] = pd.to_datetime(tmp_df['timestamp'])
        tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-05-01 22:00:00')]
        ax.plot(tmp_df['timestamp'], tmp_df['noise_width'], label=f'Pixel R{row} C{col}', marker='.', c='r' if col<8 else 'b')
        # ax.legend()
    for irun, itime in run_info.items():
        start_time = itime['start']
        end_time = start_time + datetime.timedelta(hours=itime['duration'])
        text_start_time = start_time #- datetime.timedelta(hours=1)
        text_end_time = end_time #+ datetime.timedelta(hours=1)
        plt.axvline(x=pd.to_datetime(start_time), color='magenta', linestyle='dashed')
        # ax.text(x=pd.to_datetime(text_start_time), y=400, s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
        plt.axvline(x=pd.to_datetime(end_time), color='black', linestyle='dashdot')
        # ax.text(x=pd.to_datetime(text_end_time), y=400, s=f'{irun} End', rotation=90, va='center', fontsize=12)

### Baseline in 1D hist

In [ ]:
# for idx, iboard in enumerate(interest_df['chip_name'].unique()):
#     fig, ax = plt.subplots(figsize=(10, 10))
#     hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
#     tmp_df = interest_df.loc[interest_df['chip_name']==iboard]
#     most_val = tmp_df['baseline'].mode()[0]
#     h = hist.Hist(hist.axis.Regular(20, most_val-10, most_val+10, name='test', label=f'{iboard}'))
#     h.fill(tmp_df['baseline'])
#     yerr_bool = np.array(h.values(), dtype=bool)
#     hep.histplot(h, ax=ax, yerr=yerr_bool)

### Temperature

In [ ]:
con = sqlite3.connect('/media/daq/X9/DESYJune2024/ETROC-History/ADCHistory.sqlite')
adc_df = pd.read_sql_query("SELECT * from adc", con)

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3', 'Ambient Temp']

fig, ax = plt.subplots(figsize=(20, 10))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
ax.set_title('Temperature', size=17, loc="right")

for idx, iboard in enumerate(sorted(adc_df['channel'].unique())):
    tmp_df = adc_df.loc[adc_df['channel']==iboard]
    tmp_df['timestamp'] = pd.to_datetime(tmp_df['timestamp'])
    tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-05-02 1:00:00')]
    if(idx==3): tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-05-03 20:00:00')]
    ax.plot(tmp_df['timestamp'], tmp_df['voltage'], label=f'{names[idx]}')

    if idx == 0:
        date_form = mdates.DateFormatter("%m-%d\n%H:%M")
        ax.xaxis.set_major_formatter(date_form)

for irun, itime in run_info.items():
    start_time = itime['start']
    end_time = start_time + datetime.timedelta(hours=itime['duration'])
    text_start_time = start_time #- datetime.timedelta(hours=.5)
    text_end_time = end_time #+ datetime.timedelta(hours=.25)
    plt.axvline(x=pd.to_datetime(start_time), ymin=0, ymax=1.5, color='magenta', linestyle='dashed')
    ax.text(x=pd.to_datetime(text_start_time), y=0.56, s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
    plt.axvline(x=pd.to_datetime(end_time), ymin=0, ymax=1.5, color='black', linestyle='dashdot')
    ax.text(x=pd.to_datetime(text_end_time), y=0.56, s=f'{irun} End', rotation=90, va='center', fontsize=12)
ax.legend(loc='best')
ax.set_ylabel('Voltage')
plt.ylim(bottom=0.48, top=.6)

In [ ]:
# names = ['BAR 4', 'PACTECH 1', 'BAR 5', 'BAR 12', 'Ambient Temp']
# names = ['BAR 4', 'BAR 12', 'BAR 13', 'BAR 5', 'Ambient Temp']
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3', 'Ambient Temp']
draw_text_loc = [0.57, 0.534, 0.5325, .45, .499]
draw_dates = True

for idx, iboard in enumerate(sorted(adc_df['channel'].unique())):
    fig, ax = plt.subplots(figsize=(20, 10))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
    ax.set_title(f'Board {names[idx]} Temperature', size=17, loc="right")
    # if(idx!=0): continue
    tmp_df = adc_df.loc[adc_df['channel']==iboard]
    tmp_df['timestamp'] = pd.to_datetime(tmp_df['timestamp'])
    tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-05-02 1:00:00')]
    # tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-05-06 16:50:00')]
    # tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-05-04 1:00:00')]
    # if(idx==3): tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-05-03 19:00:00')]
    tmp_df = tmp_df.loc[tmp_df['voltage']>0.3]
    ax.plot(tmp_df['timestamp'], tmp_df['voltage'], label=f'{names[idx]}')

    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)

    if(draw_dates):
        for irun, itime in run_info.items():
            start_time = itime['start']
            end_time = start_time + datetime.timedelta(hours=itime['duration'])
            text_start_time = start_time #- datetime.timedelta(hours=.5)
            text_end_time = end_time #+ datetime.timedelta(hours=.25)
            plt.axvline(x=pd.to_datetime(start_time), ymin=0, ymax=1.5, color='magenta', linestyle='dashed')
            ax.text(x=pd.to_datetime(text_start_time), y=draw_text_loc[idx], s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
            plt.axvline(x=pd.to_datetime(end_time), ymin=0, ymax=1.5, color='black', linestyle='dashdot')
            ax.text(x=pd.to_datetime(text_end_time), y=draw_text_loc[idx], s=f'{irun} End', rotation=90, va='center', fontsize=12)
            # plt.xlim(left=datetime.datetime(2024, 5, 2, 1, 0))
    ax.legend(loc='best')
    ax.set_ylabel('Voltage')
    # plt.ylim(bottom=0.48, top=.6)


In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3', 'Ambient Temp']
draw_text_loc = [20, 0.534, 0.5325, .45, .499]
draw_dates = True

for idx, iboard in enumerate(sorted(adc_df['channel'].unique())):
    if(idx!=0): continue
    fig, ax = plt.subplots(figsize=(20, 10))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
    ax.set_title(f'Board {names[idx]} Temperature', size=17, loc="right")
    tmp_df = adc_df.loc[adc_df['channel']==iboard]
    tmp_df['timestamp'] = pd.to_datetime(tmp_df['timestamp'])
    tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-06-04 12:30:00')]
    ax.plot(tmp_df['timestamp'], (402.19*tmp_df['voltage']) - 207.81, label=f'{names[idx]}')

    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)

    if(draw_dates):
        for irun, itime in run_info.items():
            start_time = itime['start']
            end_time = start_time + datetime.timedelta(hours=itime['duration'])
            text_start_time = start_time #- datetime.timedelta(hours=.5)
            text_end_time = end_time #+ datetime.timedelta(hours=.25)
            plt.axvline(x=pd.to_datetime(start_time), ymin=0, ymax=1.5, color='magenta', linestyle='dashed')
            ax.text(x=pd.to_datetime(text_start_time), y=draw_text_loc[idx], s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
            plt.axvline(x=pd.to_datetime(end_time), ymin=0, ymax=1.5, color='black', linestyle='dashdot')
            ax.text(x=pd.to_datetime(text_end_time), y=draw_text_loc[idx], s=f'{irun} End', rotation=90, va='center', fontsize=12)
    ax.legend(loc='best')
    ax.set_ylabel('Temperature [C]')
    # plt.ylim(bottom=0.48, top=.6)


### HV currents

In [ ]:
# watch python3 scripts/logNICrate.py -o May6 -d /media/daq/X9/DESYJune2024/ETROC-History/HV_logging -t 3

In [ ]:
HV_json = '/media/daq/X9/DESYJune2024/ETROC-History/HV_logging/JunTB.json'

In [ ]:
df = pd.read_json(HV_json, lines=True)

In [ ]:
# Function to convert the voltage values
def convert_voltage(series):
    series = series.str.replace(' uV', '').str.replace(' mV', '').str.replace(' V', '')
    multipliers = series.str.extract(r'(\d*\.\d+|\d+)( uV| mV| V)?', expand=True)
    multipliers[1] = multipliers[1].fillna('')
    multipliers[1] = multipliers[1].map({' uV': 1e-6, ' mV': 1e-3, ' V': 1, '': 1})
    return multipliers[0].astype('float32') * multipliers[1].astype('float32')

In [ ]:
hv_info = defaultdict(pd.DataFrame)

# Extract columns for processing
channels = df['Channel'].apply(pd.Series)
sense_voltages = df['Measured Sense Voltage'].apply(pd.Series)
terminal_voltages = df['Measured Terminal Voltage'].apply(pd.Series)
currents = df['Measured Current'].apply(pd.Series).replace(' uA', '', regex=True).astype('float32')

for inum in channels.columns:
    senseV = convert_voltage(sense_voltages[inum])
    terminalV = convert_voltage(terminal_voltages[inum])

    new_df = pd.DataFrame({
        'timestamp': pd.to_datetime(df['timestamp']),
        'senseV': senseV,
        'terminalV': terminalV,
        'currents': currents[inum]
    })

    hv_info[inum] = new_df

    del new_df

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

date_form = mdates.DateFormatter("%m-%d\n%H:%M")
ax.xaxis.set_major_formatter(date_form)

for idx, iname in enumerate(names):
    ax.plot(hv_info[idx]['timestamp'], hv_info[idx]['currents'], label=iname)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

for idx, iname in enumerate(names):
    ax.plot(hv_info[idx]['terminalV'], hv_info[idx]['currents'], label=iname)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')
ax.set_xlabel('HV Setting [V]')
ax.invert_xaxis()
ax.invert_yaxis()

In [ ]:
con = sqlite3.connect('/media/daq/X9/DESYJune2024/ETROC-History/ti1080_sensor_data.sqlite')
ti_hdc1080_df = pd.read_sql_query("SELECT * from ti_hdc1080", con)
ti_hdc1080_df['timestamp'] = pd.to_datetime(ti_hdc1080_df['timestamp'])

names = ['FBK Sensor, ET2.01 (BAR 4)','HPK Sensor, ET2.01 (BAR 12)','HPK Sensor, ET2.01 (BAR 13)','HPK Split 3 Sensor, ET2.0 (BB 1-3)']
lims  = [(11.75,9.75,11.5),(.3,.1,.175),(.3,.1,.175),(.6,.1,.2)]
idxs  = [0,1,2,3]

for idx, name, lim in zip(idxs, names, lims):

    fig, ax = plt.subplots(figsize=(30, 14))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)

    ax.plot(hv_info[idx]['timestamp'], -1*hv_info[idx]['currents'], label=iname, color='k')

    # ax.legend(loc='best')
    ax.set_ylabel('HV Currents [uA]', fontsize=25)
    ax.set_ylim(top=lim[0], bottom=lim[1])
    ax.set_xlim(left=datetime.datetime(2024, 6, 4, 19, 00), right=datetime.datetime(2024, 6, 6, 13, 0))
    ax.set_title(name, loc="right")
    if(draw_dates):
        for irun, itime in run_info.items():
            start_time = itime['start']
            if(start_time>datetime.datetime(2024, 6, 6, 13, 0)): continue
            end_time = start_time + datetime.timedelta(hours=itime['duration'])
            text_start_time = start_time #- datetime.timedelta(hours=.5)
            text_end_time = end_time #+ datetime.timedelta(hours=.25)
            plt.axvline(x=pd.to_datetime(start_time), ymin=0, ymax=1.5, color='magenta', linestyle='dashed')
            ax.text(x=pd.to_datetime(text_start_time), y=lim[2], s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
            plt.axvline(x=pd.to_datetime(end_time), ymin=0, ymax=1.5, color='black', linestyle='dashdot')
            ax.text(x=pd.to_datetime(text_end_time), y=lim[2], s=f'{irun} End', rotation=90, va='center', fontsize=12)

    ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('Onboard Temperature from VTemp [C]', color=color, fontsize=25)
    tmp_df = adc_df.loc[adc_df['channel']==sorted(adc_df['channel'].unique())[idx]]
    tmp_df['timestamp'] = pd.to_datetime(tmp_df['timestamp'])
    tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-06-04 17:00:00')]
    ax2.plot(tmp_df['timestamp'], (402.19*tmp_df['voltage']) - 207.81, label=f'{names[idx]}', color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.spines["right"].set_color(color)
    ax2.set_ylim(20, 30)

    ax3 = ax.twinx()  # instantiate a third axes that shares the same x-axis
    ax3.spines.right.set_position(("axes", 1.05))
    color = 'tab:orange'
    ax3.set_ylabel('Ambient Temperature in Suitcase [C]', color=color, fontsize=25)
    ax3.plot(ti_hdc1080_df['timestamp'], ti_hdc1080_df['temperature'], label=f'{names[idx]}', color=color, lw=3)
    ax3.tick_params(axis='y', labelcolor=color)
    ax3.spines["right"].set_color(color)
    ax3.set_ylim(20, 30)

    ax.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
names = ['BAR 4']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

date_form = mdates.DateFormatter("%m-%d\n%H:%M")
ax.xaxis.set_major_formatter(date_form)

ax.plot(hv_info[0]['timestamp'], -1*hv_info[0]['currents'], label=names[0], color='k')

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')
ax.set_ylim(top=12, bottom=6)
ax.set_title(name, loc="right")
if(draw_dates):
    for irun, itime in run_info.items():
        start_time = itime['start']
        end_time = start_time + datetime.timedelta(hours=itime['duration'])
        text_start_time = start_time #- datetime.timedelta(hours=.5)
        text_end_time = end_time #+ datetime.timedelta(hours=.25)
        plt.axvline(x=pd.to_datetime(start_time), ymin=0, ymax=1.5, color='magenta', linestyle='dashed')
        ax.text(x=pd.to_datetime(text_start_time), y=9.4, s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
        plt.axvline(x=pd.to_datetime(end_time), ymin=0, ymax=1.5, color='black', linestyle='dashdot')
        ax.text(x=pd.to_datetime(text_end_time), y=9.4, s=f'{irun} End', rotation=90, va='center', fontsize=12)

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

date_form = mdates.DateFormatter("%m-%d\n%H:%M")
ax.xaxis.set_major_formatter(date_form)

for idx, iname in enumerate(names):
    if(idx==0): continue
    ax.plot(hv_info[idx]['timestamp'], -1*hv_info[idx]['currents'], label=iname)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')

if(draw_dates):
    for irun, itime in run_info.items():
        start_time = itime['start']
        end_time = start_time + datetime.timedelta(hours=itime['duration'])
        text_start_time = start_time #- datetime.timedelta(hours=.5)
        text_end_time = end_time #+ datetime.timedelta(hours=.25)
        plt.axvline(x=pd.to_datetime(start_time), ymin=0, ymax=1.5, color='magenta', linestyle='dashed')
        ax.text(x=pd.to_datetime(text_start_time), y=.1, s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
        plt.axvline(x=pd.to_datetime(end_time), ymin=0, ymax=1.5, color='black', linestyle='dashdot')
        ax.text(x=pd.to_datetime(text_end_time), y=.1, s=f'{irun} End', rotation=90, va='center', fontsize=12)

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

for idx, iname in enumerate(names):
    if(idx==0):continue
    ax.plot(-1*hv_info[idx]['terminalV'], -1*hv_info[idx]['currents'],"--o", label=iname)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')
ax.set_xlabel('HV Setting [V]')

In [ ]:
names = ['FBK UFSD4 W15 2-3 with ETROC2.01 Chip (BAR 4)', 'BAR 12', 'BAR 13', 'BB 1-3']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

for idx, iname in enumerate(names):
    if(idx!=0):continue
    ax.plot(-1*hv_info[idx]['terminalV'][hv_info[idx]['timestamp']<datetime.datetime(2024, 6, 5, 0, 20)], -1*hv_info[idx]['currents'][hv_info[idx]['timestamp']<datetime.datetime(2024, 6, 5, 0, 20)],"--o", label=iname)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')
ax.set_xlabel('HV Setting [V]')

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

for idx, iname in enumerate(names):
    if(idx!=0):continue
    ax.plot(-1*hv_info[idx]['terminalV'], -1*hv_info[idx]['currents'], "--o", label=iname)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')
ax.set_xlabel('HV Setting [V]')
# ax.set_xlim(right=-10)
# ax.invert_xaxis()
# ax.invert_yaxis()

In [ ]:
con = sqlite3.connect('/media/daq/X9/DESYJune2024/ETROC-History/ti1080_sensor_data.sqlite')
ti_hdc1080_df = pd.read_sql_query("SELECT * from ti_hdc1080", con)
ti_hdc1080_df['timestamp'] = pd.to_datetime(ti_hdc1080_df['timestamp'])

names = ['FBK Sensor, ET2.01 (BAR 4)','HPK Sensor, ET2.01 (BAR 12)','HPK Sensor, ET2.01 (BAR 13)','HPK Split 3 Sensor, ET2.0 (BB 1-3)']
lims  = [(12,6,11.5),(.3,.1,.175),(.3,.1,.175),(.6,.1,.2)]
idxs  = [0,1,2,3]

for idx, name, lim in zip(idxs, names, lims):

    fig, ax = plt.subplots(figsize=(30, 14))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)

    ax.plot(hv_info[idx]['timestamp'], -1*hv_info[idx]['currents'], label=iname, color='k', lw=2)

    # ax.legend(loc='best')
    ax.set_ylabel('HV Currents [uA]', fontsize=25)
    ax.set_ylim(top=lim[0], bottom=lim[1])
    # ax.set_xlim(left=datetime.datetime(2024, 6, 4, 19, 00), right=datetime.datetime(2024, 6, 6, 13, 0))
    ax.set_title(name, loc="right")
    if(draw_dates):
        for irun, itime in run_info.items():
            start_time = itime['start']
            end_time = start_time + datetime.timedelta(hours=itime['duration'])
            text_start_time = start_time #- datetime.timedelta(hours=.5)
            text_end_time = end_time #+ datetime.timedelta(hours=.25)
            plt.axvline(x=pd.to_datetime(start_time), ymin=0, ymax=1.5, color='magenta', linestyle='dashed')
            ax.text(x=pd.to_datetime(text_start_time), y=lim[2], s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
            plt.axvline(x=pd.to_datetime(end_time), ymin=0, ymax=1.5, color='black', linestyle='dashdot')
            ax.text(x=pd.to_datetime(text_end_time), y=lim[2], s=f'{irun} End', rotation=90, va='center', fontsize=12)

    ax2 = ax.twinx()  # instantiate a third axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('HV Setting [V]', color=color, fontsize=25)
    ax2.plot(hv_info[idx]['timestamp'], -1*hv_info[idx]['terminalV'], color=color, lw=3)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.spines["right"].set_color(color)
    ax2.set_ylim(100, 300)
    
    ax3 = ax.twinx()  # instantiate a third axes that shares the same x-axis
    ax3.spines.right.set_position(("axes", 1.05))
    color = 'tab:orange'
    ax3.set_ylabel('Ambient Temperature in Suitcase [C]', color=color, fontsize=25)
    ax3.plot(ti_hdc1080_df['timestamp'], ti_hdc1080_df['temperature'], label=f'{names[idx]}', color=color, lw=3)
    ax3.tick_params(axis='y', labelcolor=color)
    ax3.spines["right"].set_color(color)
    ax3.set_ylim(17, 31)

    ax.grid(which="both")
    plt.tight_layout()
    plt.show()